In [62]:
# Imports
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

In [63]:
# Create Fully Connected Network
class RNN(nn.Module):
  def __init__(self, input_size, hidden_size, num_layers, num_classes):
    super(RNN, self).__init__()
    self.hidden_size = hidden_size
    self.num_layers = num_layers
    self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first = True)
    self.fc = nn.Linear(hidden_size*sequence_length, num_classes)

  def forward(self, x):
    h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
    out, _ = self.rnn(x,h0)
    out = out.reshape(out.shape[0], -1)
    out = self.fc(out)
    return out


In [64]:
# Set device

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [65]:
# Hyperparameters

input_size = 28
sequence_length = 28
num_layers = 2
hidden_size = 256
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 10

In [66]:
# Load Dataset

train_dataset = datasets.MNIST(root = '/dataset', train = True, transform = transforms.ToTensor(), download= True)
train_loader = DataLoader(dataset = train_dataset, batch_size = batch_size, shuffle = True)
test_dataset = datasets.MNIST(root = '/dataset', train = False, transform = transforms.ToTensor(), download= True)
test_loader = DataLoader(dataset = test_dataset, batch_size = batch_size, shuffle = True)

In [67]:
# Initialize Network

model = RNN(input_size, hidden_size, num_layers, num_classes).to(device)

In [68]:
# Loss and Optimizer

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr= learning_rate)

In [69]:
# Train Network

for epoch in range(num_epochs):
  print(f'Epoch {epoch} is running')
  for batch_idx, (data,targets) in enumerate(train_loader):
    # Get data to cuda
    data = data.to(device = device).squeeze(1)
    targets = targets.to(device = device)

    # Forward
    scores = model(data)
    loss = criterion(scores, targets)

    # Backword

    optimizer.zero_grad()
    loss.backward()

    # Gradient descent or adam step

    optimizer.step()

Epoch 0 is running
Epoch 1 is running
Epoch 2 is running
Epoch 3 is running
Epoch 4 is running
Epoch 5 is running
Epoch 6 is running
Epoch 7 is running
Epoch 8 is running
Epoch 9 is running


In [70]:
# Check Accuracy on training & test

def check_accuracy(loader, model):

  if loader.dataset.train:
    print('Checking accuracy on training data')
  else:
    print('Checking accuracy on testing data')
  num_correct = 0
  num_samples = 0
  model.eval()

  with torch.no_grad():
    for x,y in loader:
      x = x.to(device = device).squeeze(1)
      y = y.to(device = device)

      scores = model(x)
      _, predictions = scores.max(1)
      num_correct += (predictions == y).sum()
      num_samples += predictions.size(0)
    
    print(f'Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}')

  model.train()

check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

Checking accuracy on training data
Got 58957 / 60000 with accuracy 98.26
Checking accuracy on testing data
Got 9743 / 10000 with accuracy 97.43
